In [0]:
# In Databricks, we use scipy.optimize.linprog. This library handles the heavy lifting of the Two-Phase logic, but you must structure the matrices correctly.
import pandas as pd
import numpy as np
from scipy.optimize import linprog

# 1. Fetch the 'Critical' and 'A' items from your Gold Table
# We convert to Pandas for the mathematical solver
data = spark.table("supply_chain_opt.gold_inventory_master") \
    .filter("(priority_level = 'CRITICAL') AND (abc_category = 'A')") \
    .limit(10).toPandas()

# 2. Setup the Linear Programming components
# c: Coefficients for the objective function (Risk Scores)
c = data['stock_out_risk_score'].values * -1  # Negative because we want to maximize risk reduction

# A_ub and b_ub: Upper bound constraints (The Budget)
# [Unit Cost 1, Unit Cost 2, ...] * [x1, x2, ...] <= 10000
A_ub = [data['unit_cost'].values]
b_ub = [1000000]

# bounds: Minimum and maximum units for each item
# (Shortfall, None) ensures we order at least the required amount
bounds = [(row['reorder_point'] - row['current_stock'], None) for _, row in data.iterrows()]

# 3. Solve using the 'highs' method (which uses the Two-Phase Simplex approach)
res = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')

# 4. Process the Results
if res.success:
    data['optimized_order_qty'] = np.round(res.x)
    print("Optimal Order Plan Found!")
    display(data[['product_name', 'current_stock', 'optimized_order_qty', 'unit_cost']])
else:
    print("Status:", res.message)

Optimal Order Plan Found!


product_name,current_stock,optimized_order_qty,unit_cost
Item_29,50,147.0,236.58
Item_77,107,135.0,301.05
Item_134,44,194.0,248.2
Item_135,73,126.0,442.32
Item_171,77,132.0,290.91
Item_208,82,85.0,451.15
Item_209,41,151.0,302.84
Item_270,78,138.0,319.82
Item_278,33,60.0,368.41
Item_284,1,1558.0,405.55


In [0]:
# 1. Calculate Risk Reduction
# Risk is reduced because 'current_stock' increases, lowering the stock_out_risk_score
data['after_risk_score'] = np.round(
    (data['reorder_point'] / 30 * data['lead_time_days']) / 
    (data['current_stock'] + data['optimized_order_qty']), 2
)

# 2. Calculate Total Risk Improvement
total_before_risk = data['stock_out_risk_score'].sum()
total_after_risk = data['after_risk_score'].sum()
improvement = ((total_before_risk - total_after_risk) / total_before_risk) * 100

print(f"Total Risk Reduction: {improvement:.2f}%")

Total Risk Reduction: 89.31%


In [0]:
import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Bar(name='Before Optimization', x=data['product_name'], y=data['stock_out_risk_score'], marker_color='indianred'),
    go.Bar(name='After Optimization', x=data['product_name'], y=data['after_risk_score'], marker_color='lightseagreen')
])

fig.update_layout(
    title='Impact of Two-Phase Solver on Product Risk',
    xaxis_title='Product Name',
    yaxis_title='Stock-out Risk Score (lower is better)',
    barmode='group',
    template='plotly_white'
)

fig.show()